In [1]:
import re
import seaborn as sns
import oceanwaves
import numpy as np
import xarray as xr
from datetime import datetime

# From significant wave height to directional spectrum, and back

In [2]:
# measurement times
time     = [datetime(2013,12,1),
            datetime(2014,12,2)]

# measurement locations
location = [(0,0),(0,1),(1,1),(.5,.5)]

# initialize OceanWaves object
ow = oceanwaves.OceanWaves(time=time,
                           time_var='datetime',
                           location=location,
                           crs='epsg:28992',
                           energy_units='m',
                           energy_var='waveheight')   # note we are defining wave heights here!

ow['_energy'].values = np.random.rand(*ow.shape) * 10.    # random wave heights

# plot data and print shape and units
ow.plot()
print(ow.shape, ow.units)

(2, 4) m


In [3]:
# convert wave heights to wave spectrum with JONSWAP shape
frequency = np.arange(0,1.,.01)
ow_spc = ow.as_spectral(frequency, shape='jonswap')

# plot data and print shape and units
ow_spc.plot(col='location')
print(ow_spc.shape, ow_spc.units)

ValueError: dimensions ('datetime', 'frequency') must have the same length as the number of data dimensions, ndim=3

In [ ]:
# convert omnidirectional wave spectrum to directional wave spectrum
theta = np.arange(0., 360., 5.)
ow_dir = ow_spc.as_directional(theta, s=5)

# plot data and print shape and units
ow_dir.plot(col='location', row='datetime')
print ow_dir.shape, ow_dir.units

In [ ]:
# convert directional wave spectrum back to omnidirectional wave spectrum
ow_omn = ow_dir.as_omnidirectional()

# plot data and print shape and units
ow_omn.plot(col='location')
print ow_omn.shape, ow_omn.units

In [ ]:
# convert wave spectrum to significant wave heights
ow_Hs = ow_omn.Hm0()

# plot data and print shape and units
ow_Hs.plot()
print ow_Hs.shape, ow_Hs.units

# Other conversions

In [ ]:
# convert wave spectrum to wave period
ow_Tm02 = ow_omn.Tm02()

# plot data and print shape and units
ow_Tm02.plot()
print ow_Tm02.shape, ow_Tm02.units

In [ ]:
# convert significant wave height to directional waves
theta = np.arange(0., 360., 5.)
ow_dir2 = ow.as_directional(theta, s=5)

# plot data and print shape and units
ow_dir2.plot(col='datetime') # location is on the radials...
                             # ... and energy is still in m
print ow_dir2.shape, ow_dir2.units

# Plot spectra on map

In [ ]:
# plot spectra on a map
ow_dir[dict(datetime=0)].plot.spatial_map(scale=.3, subplot_kw=dict(projection='polar'))

# Read SWaN spectra and tables

In [ ]:
import oceanwaves

fig = plt.figure()
ow = oceanwaves.from_swan('../data/swan/P1.SP1')
ow.plot()

fig = plt.figure()
ow = oceanwaves.from_swan('../data/swan/P1.SP2')
ow.plot(col='location')

fig = plt.figure()
ow = oceanwaves.from_swantable('../data/swan/P1.TAB')
ow.plot()

# Write SWaN spectra

In [ ]:
def print_contents(fname, context=5):
    
    with open(fname, 'r') as fp:
        for line in fp:
            if re.match('[A-Z]+', line):
                n = context + 1
            if n > 0:
                print line.strip()
            if n == 0:
                print '...'
            n -= 1

In [ ]:
ow = oceanwaves.from_swantable('../data/swan/P1.TAB')
ow = ow[dict(location=range(5))]

frequency = np.arange(0,1.,.05)
ow_spc = ow.as_spectral(frequency, shape='jonswap')
ow_spc.to_swan('P1.SP1')
print_contents('P1.SP1')

In [ ]:
theta = np.arange(0., 360., 30)
ow_dir = ow_spc.as_directional(theta, s=5)
ow_dir.to_swan('P1.SP2')
print_contents('P1.SP2')

# Write netCDF

In [ ]:
ow_dir.to_netcdf('P1.nc')

# Read Datawell files

In [ ]:
ow = oceanwaves.from_datawell('../data/datawell/example.dat')
ow.plot()

# Reading WaveDroid files

In [ ]:
ow = oceanwaves.from_wavedroid('../data/wavedroid/example.dat')
ow.plot()

# Unit conversion

In [ ]:
print oceanwaves.units.simplify('m^2 / m')
print oceanwaves.units.simplify('m ^ 2 / m')                               # test spaces
print oceanwaves.units.simplify('m2 / m')                                  # test missing exponents
print oceanwaves.units.simplify('m^3 / (m * m)')                           # test groups
print oceanwaves.units.simplify('m^4 / (m * (m / m^-1))')                  # test nested groups
print oceanwaves.units.simplify('m^5 / (m * m)^2')                         # test group exponents
print oceanwaves.units.simplify('m^5 / (m * m)^2 + NAP')                   # test terms
print oceanwaves.units.simplify('(kg * m^2 * s^2)^2 / (kg^2 * m^3 * s^4)') # multiple units
print oceanwaves.units.simplify('(m^2 / Hz) * m^-2')                       # hertz to seconds conversion

In [ ]:
import json
import urllib2

r = urllib2.urlopen('https://circleci.com/api/v1.1/project/github/openearth/oceanwaves-python/latest/artifacts')
r.read()